In [ ]:
import matplotlib.pyplot as plt
import os
import random
import shutil
import numpy as np
import pandas as pd
import skimage.transform as sktf
from PIL import Image

### Download data

In [ ]:
%%bash

export KAGGLE_USERNAME='hung1578'
export KAGGLE_KEY='aa415e7d1ab043d0afe78e1dce04211d'
kaggle competitions download -c h-and-m-personalized-fashion-recommendations

In [ ]:
%unzip /content/h-and-m-personalized-fashion-recommendations.zip

In [ ]:
%rm /content/h-and-m-personalized-fashion-recommendations.zip

### Read data

In [ ]:
articles = pd.read_csv("articles.csv", dtype={"article_id": "str"})
customers = pd.read_csv("customers.csv")
transactions = pd.read_csv("transactions_train.csv", dtype={"article_id": "str"})

### Sample data

In [ ]:
sample = 0.001
customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False, random_state=45)
customers_sample_ids = set(customers_sample["customer_id"])
transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
articles_sample_ids = set(transactions_sample["article_id"])

In [ ]:
articles_id_ghost = []
print(len(customers_sample_ids),len(articles_sample_ids))
for i in articles_sample_ids:
    if not os.path.exists(f'images/{i[:3]}/{i}.jpg'):
        articles_id_ghost.append(i)

In [ ]:
transactions_sample = transactions_sample.loc[~transactions_sample["article_id"].isin(articles_id_ghost)]

In [ ]:
articles_sample_ids = set(transactions_sample["article_id"])
articles_id_ghost = []
print(len(customers_sample_ids),len(articles_sample_ids))
for i in articles_sample_ids:
    if not os.path.exists(f'images/{i[:3]}/{i}.jpg'):
        articles_id_ghost.append(i)
articles_id_ghost

In [ ]:
if os.path.exists('data/images'):
    shutil.rmtree('data/images')
    os.makedirs('data/images')

In [ ]:
articles_sample_ids = set(transactions_sample["article_id"])
articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
customers_sample.to_csv("data/customers_sample.csv.gz", index=False)
transactions_sample.to_csv("data/transactions_train_sample.csv.gz", index=False)
articles_sample.to_csv("data/articles_train_sample.csv.gz", index=False)

In [ ]:
for i in articles_sample_ids:
    if not os.path.exists(f'data/images/{i[:3]}/'): 
        os.mkdir(f'data/images/{i[:3]}/')

    im = plt.imread(f'images/{i[:3]}/{i}.jpg')
    im = sktf.resize(im,(540,360,3),mode='constant',preserve_range=True)
    im = (im-im.min())/(im.max()-im.min()) * 255.0
    im = im.astype(np.uint8)

    im = Image.fromarray(im)
    im.save(f'data/images/{i[:3]}/{i}.jpg')